# Set Working Directory

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


# Preprocessing data (Default when repository is given)

## Create GRAPHRAG DB / Preprocess and insert Data

In [7]:
from tools.micro_process import ProcessMicroData
from langchain_anthropic import ChatAnthropic
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o", model_provider="openai")
# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

gen_graphdb = ProcessMicroData(llm=llm)
graph_db, graph_documents = gen_graphdb.analyze_codebase(repo_path='../docs/notebooks/assignment-2-mcdonald-s/src')

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src\tools\micro_process.py:216: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  repo_tree_string = generate_repo_tree(repo_path)


Processing file: deduplication\bloom_filter.py
Processing file: deduplication\dedup.py
Processing file: deduplication\LSH.py
Processing file: deduplication\LSHForest.py
Processing file: deduplication\LSHImproved.py
Processing file: deduplication\__init__.py
Processing file: deduplication\__main__.py
Processing file: utils\use_cases.py
Processing file: utils\utils.py
Processing file: utils\visualizations.py
Processing file: utils\visualization_lsh.py
Converting to graph documents...
Adding graph documents to Neo4j...


### Reset Instance

In [9]:
# graph_db.query("MATCH (n) DETACH DELETE n")

## Visualize Entire Graph

In [4]:
a = gen_graphdb.create_visualization(graph_documents = graph_documents)

Graph saved as semantic_graph.html


## Create Vector DB / Preprocess and insert Data

In [7]:
from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="claude-3-5-sonnet-20240620",
)


# Demonstrate Tools

## Cypher Query to generate initial Answer

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
# from langchain_core.language_models import BaseChatModel
# from langchain_neo4j import Neo4jGraph
# from tools.micro_cypher_chain import CypherGraphBuilder

# url = os.getenv('NEO4J_URI')
# username = os.getenv('NEO4J_USERNAME')
# password = os.getenv('NEO4J_PASSWORD')

# graph_db = Neo4jGraph(
#     url=url,
#     username=username,
#     password=password,
#     enhanced_schema=True
#     )

# def retrieve_cypher_relationships(question: str):
#     """
#     Connects to a local Neo4j instance and uses a language model to generate a Cypher-based graph
#     representation of relationships between entities (e.g., code modules).

#     Returns:
#         dict: A structured representation of nodes and relationships inferred from the graph.
#     """

#     # Create the Cypher graph
#     builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
#     # cypher_graph = builder.create_cypher_graph()
#     cypher_graph = builder.create_cypher_graph()
#     result = cypher_graph.invoke({"question": question})

#     return result['answer']

In [3]:
from langchain_neo4j import Neo4jGraph
from tools.micro_cypher_chain import CypherGraphBuilder
from langchain_anthropic import ChatAnthropic

url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    enhanced_schema=True
)


# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")


builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
cypher_graph = builder.create_cypher_graph()

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done
done


In [4]:
result = cypher_graph.invoke({"question": "How is the Utils.Utils module related to the Deduplication.__Main__ module?"})

In [5]:
result['answer']

'The `Utils.Utils` module is related to the `Deduplication.__Main__` module in the following ways:\n\n- **Uses Minhash**: `Utils.Utils` imports `Utils.Utils.Minhash`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Uses Shingle**: `Utils.Utils` imports `Utils.Utils.Shingle`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Uses Clean Document**: `Utils.Utils` imports `Utils.Utils.Clean_Document`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Reads TSV**: `Utils.Utils` defines `Utils.Utils.Read_Tsv`, which is used in `Deduplication.__Main__`.\n- **Used in**: Definitions from `Utils.Utils` that involve `Utils.Utils.Read_Tsv` are specifically indicated as being used in `Deduplication.__Main__` and `Deduplication.__Main__.Main`.'

In [6]:
result['cypher_results']

[{'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Minhash'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'IMPORTED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Minhash'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'USED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Shingle'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'IMPORTED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind

In [7]:
# graph_db.query("""
# MATCH path = (m1:Module {id: 'Utils.Utils'})-[*..5]-(m2:Module {id: 'Deduplication.__Main__'}) 
# WHERE NONE(n IN nodes(path) WHERE n:Package)
# RETURN path
# """)

In [13]:
# import os
# print(os.getcwd())
# os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
# print(os.getcwd())

# from tools.micro_tools import (retrieve_cypher_relationships)

# cypher_generator = retrieve_cypher_relationships("How is the Utils.Utils module related to the Deduplicatioin.__Main__ module?")

## Final Text Response(VectorDB)

In [9]:
questions = """
The Utils.Utils module is intricately related to the Deduplication.__Main__ module through several intermediate connections, reflecting a well-structured framework that enhances document deduplication capabilities. Here are the key points of this relationship:

1. **Function Utilization**: The Utils.Utils module utilizes the `Compute_Minhash_Signatures` function from Deduplication.Lsh.Lsh, which plays a crucial role in generating MinHash signatures for documents. This computational function is essential for the efficiency of the deduplication process, as it allows for quick similarity comparisons between documents based on their hashed representations.

2. **Class Interdependency**: The Deduplication.Lsh.Lsh module invokes the `Model` class from Deduplication.__Main__. This class serves as the backbone of the deduplication system, encapsulating core functionalities and configurations necessary for implementing various LSH-based methods effectively. The relationship emphasizes a modular design where the Lsh module extends the capabilities defined in the __Main__ module, facilitating code maintainability and scalability.

3. **Definition of Key Classes**: The Deduplication.__Main__ module independently defines both the `Model` class and the `__Main__` class. The `Model` class encompasses methods for executing the deduplication algorithms and managing the data structures involved, while the `__Main__` class is responsible for orchestrating the overall execution flow of the deduplication process. This design fosters a clear separation of concerns, allowing more straightforward debugging and enhancement in future iterations.

4. **Indirect Connections through Variants**: Additional relationships exist through other modules like `Lshimproved.Lsh` and `Lshforest.Lshforest`, which are specifically imported by Deduplication.__Main__. These imports signify that the main deduplication script leverages advancements in LSH techniques, such as improved processes for handling larger datasets and enhanced accuracy in identifying duplicate documents.

In summary, the Utils.Utils module is primarily connected to Deduplication.__Main__ through shared components and dependencies within the overall Deduplication system. These connections enhance the utility and efficiency of the deduplication process, particularly through the Lsh module and its associated classes and functions. The architecture aligns with best practices in software development by promoting modularity, enabling future enhancements like the implementation of new LSH-based algorithms without extensive modifications to the existing structure. This design not only streamlines the deduplication process but also positions the system for potential application in various scenarios, including plagiarism detection and content clustering.
"""
# Query the system
answer = query_engine.query(result['answer'])
print(answer)

Based on the provided sentence and context, I can add relevant details to make it more informative and useful:

The `Utils.Utils` module is a crucial component in the `Deduplication.__Main__` module, providing essential functionality for document processing and deduplication. Specifically:

1. The `Utils.Utils` module offers several key functions that are utilized in `Deduplication.__Main__`:

   - `Read_Tsv`: This function reads tab-separated values (TSV) files, likely used to load document datasets for deduplication.
   - `Minhash`: Implements the MinHash algorithm to create compact signatures of document shingles, essential for efficient similarity estimation between documents.
   - `Shingle`: Generates k-shingles from text, preserving some word order information for document comparison.
   - `Clean_Document`: Normalizes text by converting to lowercase and removing non-alphabetic characters, crucial for preprocessing before analysis.
   - `Majority_Vote`: Performs a majority vote on

## Final Visualization Response

In [10]:
from tools.micro_visual_response import visualize_cypher_results
stats = visualize_cypher_results(result['cypher_results'], "code_relationships_graph.html")

Graph visualization saved as code_relationships_graph.html


# Agent

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
from tools.micro_tools import (retrieve_cypher_relationships, visualize_relationships, generate_text_response)
from tools import micro_tools
from langgraph.prebuilt import create_react_agent

micro_tools.query_engine = query_engine

tools = [
    retrieve_cypher_relationships,
    visualize_relationships,
    generate_text_response,
]

system_prompt = f"""
You are a suupervisor that answers the questions given the following tools.
{tools}

You will use each tool only once and the guidelines of the tools are as follows:
- You will always use the retrieve_cypher_relationships tool to get an initial answer.
- You will always use the generate_text_response tool to get a final answer.
- Only use the visualize_relationships tool to visualize the relationships when the user asks for a visualization.

USE EACH TOOL ONLY ONCE.

Always provide clear, actionable responses with specific details and implementation steps.
"""

from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o", model_provider="openai")

agent = create_react_agent(llm, tools=tools, prompt=system_prompt)

In [3]:
answer = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "How is the Utils.Utils module related to the Deduplication.__Main__ module?",
            }
        ],
        "goto": "supervisor",
    }
)

importing tools


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


initializing Neo4jGraph
Building Cypher graph
done
done
Invoking cypher graph
Generating text response...
Results: {'answer': '- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is imported into `Deduplication.Lshforest`, eventually making its way into `Deduplication.__Main__`.\n- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is used in `Deduplication.Lshforest.Lshforest.Banding`, and this is imported into `Deduplication.__Main__`.\n- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is part of `Deduplication.Lshforest.Lshforest`, extending `Deduplication.Lsh.Lsh` that is imported into `Deduplication.__Main__`.\n- `Utils.Utils` directly defines content (`Utils.Utils.Majority_Vote`) that is used by `Deduplication.__Main__`, specifically in its model usage.\n- `Utils.Utils.Read_Tsv` is defined in `Utils.Utils` and imported into `Deduplication.__Main__`.', 'steps': ['generate_cypher', 'execute_cypher', 'generate_final_answer'], 'cypher_statement': "MATCH path = (

In [4]:
print(answer['messages'][-1].content)

The `Utils.Utils` module and the `Deduplication.__Main__` module are connected through several relationships and dependencies within the codebase. Here is a detailed breakdown of their relationship:

1. **Definition and Usage**:
   - `Utils.Utils` defines a component called `Utils.Utils.Majority_Vote`. This component is crucial and is imported into various parts of the deduplication framework, including `Deduplication.Lshforest` and through to `Deduplication.__Main__`.
   - `Utils.Utils.Majority_Vote` is specifically used within `Deduplication.Lshforest.Lshforest.Banding`, which is part of the operations in the `Deduplication.__Main__`.

2. **Hierarchical Integration**:
   - The component `Utils.Utils.Majority_Vote` is integrated into `Deduplication.Lshforest`. The module `Deduplication.Lshforest` contains LSH (Locality-Sensitive Hashing) functionalities that extend its capabilities via the integration of majority voting systems for enhanced duplicate detection or elimination.
   - The

In [4]:
answer

{'messages': [HumanMessage(content='How is the Utils.Utils module related to the Deduplication.__Main__ module? Can you give me a visualization as well?', additional_kwargs={}, response_metadata={}, id='93292251-b958-4aaa-acda-f0956832283b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xumAZ62KiNB87z21VqqKcaU5', 'function': {'arguments': '{"question":"How is the Utils.Utils module related to the Deduplication.__Main__ module?"}', 'name': 'retrieve_cypher_relationships'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 731, 'total_tokens': 766, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_85bc8e3fae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-56e374d2-9391-4c

In [6]:
query_engine.query('utils')

NotFoundError: Error code: 404 - {'error': {'message': 'The model `claude-3-5-sonnet-20240620` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

# Entire Agent Demo

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)

In [9]:
from agent_router import create_router_graph
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chat_models import init_chat_model

from tools.code_analysis_tools import (explain_code_logic, generate_repo_tree, list_python_files, read_code_file)
import  tools.micro_tools as mt

import importlib
importlib.reload(mt)

# import (retrieve_cypher_relationships, visualize_relationships, generate_text_response)

llm = init_chat_model("gpt-4o", model_provider="openai")

macro_tools = [
    # generate_repo_tree,
    # read_code_file,
    # list_python_files,
    # explain_code_logic,
]

# Add micro tools if available
mt.query_engine = query_engine

micro_tools = [
    mt.retrieve_cypher_relationships,
    mt.visualize_relationships,
    mt.generate_text_response,
]
graph = create_router_graph(
            llm=llm, macro_tools=macro_tools, micro_tools=micro_tools
        )



2025-04-18 09:59:54 - agent_router - INFO - Building the router graph...


INFO:agent_router:Building the router graph...


2025-04-18 09:59:54 - agent_router - INFO - Compiling the router graph...


INFO:agent_router:Compiling the router graph...


In [10]:
full_query = 'How is the Utils.Utils module related to the Deduplication.__Main__ module?'

# Add the user message to the conversation history
user_message = HumanMessage(content=full_query)

# Initialize the state with the user query
initial_state = {"messages": [user_message]}
result = graph.invoke(initial_state)

2025-04-18 09:59:55 - agent_router - INFO - Supervisor decided to route to: micro


INFO:agent_router:Supervisor decided to route to: micro


2025-04-18 09:59:55 - agent_router - INFO - Micro agent is processing...


INFO:agent_router:Micro agent is processing...


2025-04-18 10:00:03 - agent_router - INFO - Tool Call: Cypher query generated -> {'answer': 'The `Utils.Utils` module is related to the `Deduplication.__Main__` module through the following connections:\n\n- `Utils.Utils` imports `Utils.Use_Cases`, which defines `Nearest_Neighbor_Search`, imported into `Deduplication.__Main__`.\n- `Utils.Utils` imports `Utils.Use_Cases`, defining `Collection_Deduplication`, imported into `Deduplication.__Main__`.\n- `Utils.Utils` imports `Utils.Utils.Unionfind`, which uses `Collection_Deduplication`, imported into `Deduplication.__Main__`.\n- `Utils.Utils` imports `Utils.Utils.Unionfind`, used in defining `Utils.Use_Cases` and `Nearest_Neighbor_Search`, imported into `Deduplication.__Main__`.\n- `Utils.Utils` imports `Utils.Utils.Unionfind`, defined and used in `Deduplication.Lsh`, imported into `Deduplication.__Main__`.', 'steps': ['generate_cypher', 'execute_cypher', 'generate_final_answer'], 'cypher_statement': "MATCH path = (m1:Module {id: 'Utils.U

...

INFO:agent_router:Generating text response...


2025-04-18 10:00:16 - agent_router - INFO - Query engine: <tools.micro_text_answer.RAGQueryEngine object at 0x000002A48C6AFB60>


INFO:agent_router:Query engine: <tools.micro_text_answer.RAGQueryEngine object at 0x000002A48C6AFB60>


2025-04-18 10:00:33 - agent_router - INFO - Supervisor decided to route to: FINISH


INFO:agent_router:Supervisor decided to route to: FINISH


In [8]:
result['messages']

[HumanMessage(content='How is the Utils.Utils module related to the Deduplication.__Main__ module?', additional_kwargs={}, response_metadata={}, id='be497bad-2fbe-4aac-b6af-3728813cfb60'),
 AIMessage(content="I've decided to route to micro because this is the appropriate next step.", additional_kwargs={}, response_metadata={}, name='supervisor', id='769fa0d7-5bdb-419a-941c-f00308bf3db5'),
 AIMessage(content='The `Utils.Utils` module and the `Deduplication.__Main__` module have the following relationships:\n\n1. **Import and Dependency**: \n   - The `Utils.Utils` module directly imports `Utils.Use_Cases`, which defines important components like `Nearest_Neighbor_Search` and `Collection_Deduplication`. These components are crucial for deduplication functions and are ultimately imported into `Deduplication.__Main__`.\n   \n2. **Indirect Utilization**:\n   - The `Utils.Utils` module also imports `Utils.Utils.Unionfind`, which is instrumental for handling collections and is used by `Collect

In [19]:
print(result['messages'][-2].content)

The `Utils.Utils` module is intricately linked with the `Deduplication.__Main__` module, primarily through import and utility sharing, playing a pivotal role in document deduplication and similarity processes. Here's a detailed explanation:

1. **Direct Import Connections**:
   - **Utility Modules**: `Utils.Utils` imports `Utils.Use_Cases`, which defines key functionalities such as `Nearest_Neighbor_Search` and `Collection_Deduplication`. These are directly leveraged by `Deduplication.__Main__` to perform core tasks in deduplication processes.
   - **Data Structures**: The `Utils.Utils.Unionfind` is not only imported into `Deduplication.__Main__` but is also essential for performing clustering operations within a dataset, highlighting shared utility across the modules.

2. **Functionality Extensions**:
   - The utilities like `Minhash`, `Shingle`, and `Clean_Document` extend the capabilities of `Nearest_Neighbor_Search` by providing preprocessing steps essential for accurate similarity

In [ ]:
    if "graph" not in st.session_state:
        #########
        # MODEL #
        #########
        # llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
        from langchain.chat_models import init_chat_model
        llm = init_chat_model("gpt-4o", model_provider="openai")

        #########
        # TOOLS #
        #########
        tools = setup_tools()

        # Make sure tools have access to the database if it exists
        if "database" in st.session_state and "query_engines" in st.session_state.database:
            try:
                from tools import micro_tools
                first_engine_key = list(st.session_state.database["query_engines"].keys())[0]
                query_engine = st.session_state.database["query_engines"][first_engine_key]
                micro_tools.query_engine = query_engine
            except Exception as e:
                logger.warning(f"Could not initialize micro tools with query engine: {e}")

        st.session_state.graph = create_router_graph(
            llm=llm, macro_tools=tools["macro_tools"], micro_tools=tools["micro_tools"]
        )